In [1]:
import sys
print(sys.path)
sys.path.append('/Users/leehojin/Projects/new_automation/')

['/opt/homebrew/Caskroom/miniconda/base/lib/python313.zip', '/opt/homebrew/Caskroom/miniconda/base/lib/python3.13', '/opt/homebrew/Caskroom/miniconda/base/lib/python3.13/lib-dynload', '', '/Users/leehojin/Projects/new_automation/.venv/lib/python3.13/site-packages']


In [2]:
from src.crawler import Crawler
from playwright.async_api import async_playwright
import logging

#로깅
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

In [3]:
pw = await async_playwright().start()
browser = await pw.chromium.launch(
    headless=False,  # 화면 켜기
    slow_mo=500,
)
page = await browser.new_page()
crawler = Crawler(page)

In [4]:
await crawler.login.login()

2025-12-02 17:03:20,253 - src.crawler.login - INFO - 로그인 시작
2025-12-02 17:03:22,820 - src.crawler.login - INFO - 로그인 성공


In [5]:
await crawler.popup.close_all_popups()

2025-12-02 17:03:29,415 - src.crawler.popup_handler - INFO - 팝업 닫기 성공: button.swal2-confirm
2025-12-02 17:03:30,466 - src.crawler.popup_handler - INFO - 팝업 닫기 성공: .guide-modal-close
2025-12-02 17:03:30,998 - src.crawler.popup_handler - INFO - 총 2개의 팝업을 닫았습니다.


2

In [6]:
await crawler.nav.open_sidebar()

In [7]:
await crawler.nav.click_police_button()

In [22]:
filtered_data = await crawler.filter.filtered()
print(filtered_data)

[{'id': '4', 'type': '1', 'fbUid': '0EHWPXDKeWcNN2UOSbwILpVbCqs1', 'nick': 'zuni', 'country': 'VN', 'gender': 'female', 'lastLogin': '2025. 12. 2. 오후 3:47:39', 'captureLink': 'http://admin.gobermuda.net:8080/userCapture?fbUid=0EHWPXDKeWcNN2UOSbwILpVbCqs1'}, {'id': '7', 'type': '1', 'fbUid': '0M0BbO1VgpUL99tHIXxHtxuFk182', 'nick': 'Giselle', 'country': 'VN', 'gender': 'female', 'lastLogin': '2025. 12. 1. 오후 6:56:47', 'captureLink': 'http://admin.gobermuda.net:8080/userCapture?fbUid=0M0BbO1VgpUL99tHIXxHtxuFk182'}, {'id': '8', 'type': '1', 'fbUid': '0MuzG639jiPE0EPC8Hh8aSUVtqA2', 'nick': 'Gabi', 'country': 'VE', 'gender': 'female', 'lastLogin': '2025. 11. 25. 오전 3:26:03', 'captureLink': 'http://admin.gobermuda.net:8080/userCapture?fbUid=0MuzG639jiPE0EPC8Hh8aSUVtqA2'}, {'id': '11', 'type': '1', 'fbUid': '0TKHGs3bpJh1glPpojM5OAm8Qc53', 'nick': 'avni', 'country': 'IN', 'gender': 'female', 'lastLogin': '2025. 12. 2. 오후 1:46:58', 'captureLink': 'http://admin.gobermuda.net:8080/userCapture?fbUi

In [37]:
import os
import re
import aiohttp
from datetime import datetime
from playwright.async_api import Page

async def save_all_imgs_flat(page: Page, folder_name: str):
    save_dir = f"src/test/image/{folder_name}"
    os.makedirs(save_dir, exist_ok=True)

    imgs = page.locator("img")
    count = await imgs.count()
    print(f"▶ 전체 이미지: {count}장")

    async with aiohttp.ClientSession() as session:
        for i in range(count):
            src = await imgs.nth(i).get_attribute("src")
            if not src:
                continue

            file_path = f"{save_dir}/img_{i+1}.jpg"

            async with session.get(src) as res:
                content = await res.read()
                with open(file_path, "wb") as f:
                    f.write(content)

            print(f"  ✔ 저장: {file_path}")


async def save_images_by_date(page: Page, row: dict):
    """
    filtered_data 에서 row를 전달
    """
    fbUid = row["fbUid"]
    nick = row["nick"]
    country = row["country"]
    gender = row["gender"]

    folder_name = f"{fbUid}_{nick}_{country}_{gender}"
    folder_name = re.sub(r'[^a-zA-Z0-9가-힣_]', '', folder_name)

    print(f"\n=== [{fbUid}] 캡처 페이지 진입 ===")

    # 반드시 "클릭으로 새창 열기"
    async with page.context.expect_page() as popup:
        await page.click(f"a[href*='{fbUid}']")

    new_page = await popup.value
    await new_page.wait_for_load_state("networkidle")

    # 날짜 섹션 탐색
    sections = new_page.locator(".date-photo-data")
    section_count = await sections.count()

    print(f"▶ 날짜 섹션: {section_count}개")

    # 날짜 섹션 없는 경우 (전체 이미지 저장)
    if section_count == 0:
        print("▶ 날짜 정보 없음 → 전체 이미지 저장")
        await save_all_imgs_flat(new_page, folder_name)
        return

    async with aiohttp.ClientSession() as session:
        for i in range(section_count):
            section = sections.nth(i)

            date_id = await section.get_attribute("id")
            if not date_id:
                continue
            
            # 날짜 포맷 처리
            try:
                date_folder = datetime.strptime(date_id, "%Y%m%d").strftime("%Y-%m-%d")
            except ValueError:
                date_folder = date_id

            # 이미지 찾기
            imgs = section.locator("img")
            img_count = await imgs.count()
            print(f"▶ {date_folder}: {img_count}장")

            # 이미지 없으면 폴더 생성 안함
            if img_count == 0:
                print(f"→ {date_folder}: 스킵")
                continue

            save_dir = f"src/test/image/{folder_name}/{date_folder}"
            os.makedirs(save_dir, exist_ok=True)

            # 다운로드
            for n in range(img_count):
                src = await imgs.nth(n).get_attribute("src")
                if not src:
                    continue

                file_path = f"{save_dir}/img_{n+1}.jpg"

                async with session.get(src) as res:
                    content = await res.read()
                    with open(file_path, "wb") as f:
                        f.write(content)

                print(f"  ✔ 저장: {file_path}")

    print(f"=== [{fbUid}] 완료 ===")


In [40]:
await save_images_by_date(page, filtered_data[158])


=== [LM9JO2yuOXXgx5b7saBNZJB818o2] 캡처 페이지 진입 ===
▶ 날짜 섹션: 8개
▶ 2025-12-02: 11장
  ✔ 저장: src/test/image/LM9JO2yuOXXgx5b7saBNZJB818o2_muazez_TR_female/2025-12-02/img_1.jpg
  ✔ 저장: src/test/image/LM9JO2yuOXXgx5b7saBNZJB818o2_muazez_TR_female/2025-12-02/img_2.jpg
  ✔ 저장: src/test/image/LM9JO2yuOXXgx5b7saBNZJB818o2_muazez_TR_female/2025-12-02/img_3.jpg
  ✔ 저장: src/test/image/LM9JO2yuOXXgx5b7saBNZJB818o2_muazez_TR_female/2025-12-02/img_4.jpg
  ✔ 저장: src/test/image/LM9JO2yuOXXgx5b7saBNZJB818o2_muazez_TR_female/2025-12-02/img_5.jpg
  ✔ 저장: src/test/image/LM9JO2yuOXXgx5b7saBNZJB818o2_muazez_TR_female/2025-12-02/img_6.jpg
  ✔ 저장: src/test/image/LM9JO2yuOXXgx5b7saBNZJB818o2_muazez_TR_female/2025-12-02/img_7.jpg
  ✔ 저장: src/test/image/LM9JO2yuOXXgx5b7saBNZJB818o2_muazez_TR_female/2025-12-02/img_8.jpg
  ✔ 저장: src/test/image/LM9JO2yuOXXgx5b7saBNZJB818o2_muazez_TR_female/2025-12-02/img_9.jpg
  ✔ 저장: src/test/image/LM9JO2yuOXXgx5b7saBNZJB818o2_muazez_TR_female/2025-12-02/img_10.jpg
  ✔ 저장: src/test/im